In [1]:
import pandas as pd

In [56]:
data_df = pd.read_json("CMeEE_train.json")

In [68]:
data_df = pd.read_json("CMeEE_dev.json")

In [69]:
def _split_long_sent(query_data: list, max_seq_len=128, ratio_slide=1):
    """
    This function is for split the sentence which length over 128
    :param query_data: list of string
    :return: processed_query_data: list of string, length of each string smaller than 128
             concate_data_dict: map of raw text to splitted data
    """
    processed_query_data = []
    processed_seq_idx = []

    query_len = len(query_data)
    concate_data_dict = dict(zip(range(query_len), [[] for _ in range(query_len)]))

    for idx_sent, sent in enumerate(query_data):
        sent_len = len(str(sent))
        if sent_len > max_seq_len:  # parse the sentence if length larger than max len
            split_pos = 0
            while sent_len - split_pos > max_seq_len:  # if the rest part shorter than max seq_len, break
                is_split = False  # flag, if sentence does not contain any split symbol will be directly appended
                cur_head = split_pos + max_seq_len - 1

                alter_split_pos = None

                for idx, char in enumerate(sent[cur_head:split_pos:-1]):
                    if char in [",", ".", ";", "，", "。", "；"]:
                        processed_query_data.append(sent[split_pos:cur_head - idx + 1])
                        processed_seq_idx.append([split_pos, cur_head - idx])

                        split_pos += max_seq_len - idx
                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                        is_split = True
                        break

                    if (not alter_split_pos) and char in [")","）"]:
                        alter_split_pos = idx

                if not is_split:  # full sent added into list

                    if alter_split_pos is not None:
                        processed_query_data.append(sent[split_pos:cur_head - alter_split_pos + 1])
                        processed_seq_idx.append([split_pos, cur_head - alter_split_pos])

                        split_pos += max_seq_len - alter_split_pos
                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                    else:
                        print(query_data)
                        processed_query_data.append(sent[split_pos:split_pos + max_seq_len])
                        processed_seq_idx.append([split_pos, split_pos + max_seq_len - 1])

                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                        split_pos += max_seq_len

                slide_pos = int(split_pos - (1 - ratio_slide) * len(processed_query_data[-1]))
                for idx, char in enumerate(sent[slide_pos:split_pos]):
                    if char in [",", ".", ";", "，", "。", "；",")","）"]:
                        split_pos = idx + slide_pos + 1
                        break

            processed_query_data.append(sent[split_pos:])
            processed_seq_idx.append([split_pos, sent_len - 1])

        else:
            processed_query_data.append(sent)
            processed_seq_idx.append([0, sent_len - 1])

        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)  # record the last span or short sentence

    return processed_query_data, concate_data_dict, processed_seq_idx

In [70]:
new_data=list()
for item in data_df.itertuples():
    region_dict = dict()
    entity_list = list()

    for entity in item.entities:
        start = entity["start_idx"]
        end = entity["end_idx"]
        region_dict[(start, end)] = {"type": entity["type"], "entity": entity["entity"]}

    seq_list, seq_dict, seq_idx_list = _split_long_sent([item.text], ratio_slide=0.5)


    for seq, seq_idx in zip(seq_list, seq_idx_list):
        seq_len = len(seq)
        sub_seq_region_list = list()
        for (start, end), entity in region_dict.items():
            if end <= seq_idx[1] and start >= seq_idx[0]:
                sub_seq_entity = {"start_idx": start - seq_idx[0], "end_idx": end - seq_idx[0], "type": entity["type"],
                                  "entity": entity["entity"]}
                sub_seq_region_list.append(sub_seq_entity)

                if entity["entity"] != seq[sub_seq_entity["start_idx"]:sub_seq_entity["end_idx"] + 1]:
                    print("error")

        new_data.append({"text":seq,"entities":sub_seq_region_list})


['参考文献1.李正，王慧贞，吉士俊.先天畸形学.北京：人民卫生出版社，20002.王忠诚.神经外科学.武汉：湖北科学技术出版社，19983.李世绰，程学铭，王文志，等.神经系统疾病流行病学.北京：人民卫生出版社，20004.肖坤则，张芝燕，高健，等.中国神经管缺陷的流行病学.中华医学杂志，1989，69（4）：189-1895.陈晓玲，戴晓瑛，刘建涛，等.叶酸与神经管畸形.河南预防医学杂志，1999，10（5）：326-3266.BarberRC，LammerEJ，ShawGM，etal.Theroleoffolatetransportandmetabolisminneuraltubedefectrisk.MolGenetMetab，1999，66（1）：1-17.AntonyAC，HansenDK.Hypothesis：folate-responsiveneuraltubedefectsandneurocristopathies.Teratology，2000，62（1）：42-428.McCombJG.Spinalandcranialneuraltubedefects.SeminPediatrNeurol，1997，4（3）：156-1569.CsabayL，SzaboI，PappC，etal.Centralnervoussystemanomalies.AnnNYAcadSci，1998，847：21-4510.CzeizelAE.Primarypreventionofneural-tubedefectsandsomeothermajorcongenitalabnormalities：recommendationsfortheappropriateuseoffolicacidduringpregnancy.PaediatrDrugs，2000，2（6）：437-44911.EskesTK.Openorclosed？']
['参考文献1.沈晓明.临床儿科学.北京：人民卫生出版社，2005年：225-2252.岳少杰.新生儿细菌性脑膜炎诊断中的几个问题.中国实用儿科杂志，2011，26（1）：11-113.余佳.化脓性脑膜炎的临床研究进展.儿科药学杂志，2006，12（3）：61-634.王艺，王传清，王晓红.细菌性脑膜炎266例病原学与耐药性分析.实用儿科临床杂

In [71]:
# check
for item in pd.DataFrame(new_data).itertuples():
    for entity in item.entities:
        if item.text[entity["start_idx"]:entity["end_idx"]+1] != entity["entity"]:
            print("error")

In [72]:
processed_data = []
for item in pd.DataFrame(new_data).itertuples():
    row_data_dict = dict()
    # insert space between every two chars
    row_data_dict["text_a"] = " ".join(list("".join((item.text.split()))))

    # generate sentence label
    label_sent = ["O"] * (len(item.text))
    region_dict = dict()

    for entity in item.entities:
        start = entity["start_idx"]
        end = entity["end_idx"]

        if start == end:
            label_sent[start] = "S"
            region_dict[(start, end)] = entity["type"]
            continue

        if label_sent[start] == "E" or label_sent[end] == "B":
            print(item, "boundary redandency")
        else:
            label_sent[start] = "B" if label_sent[start] != "S" else "S"
            label_sent[end] = "E" if label_sent[end] != "S" else "S"

            region_dict[(start, end)] = entity["type"]

            for ele_idx, elem_label in enumerate(label_sent[start + 1:end]):
                if elem_label == "O":
                    label_sent[ele_idx + start + 1] = "M"

    row_data_dict["bd_label"] = " ".join(label_sent)

    # generate region label
    label_region = []
    for start_idx, head in enumerate(label_sent):
        if head == "S" or head == "B":
            # single entity
            if head == "S":
                label_region.append(region_dict[(start_idx, start_idx)])

            # other entity
            for end_idx, tail in enumerate(label_sent[start_idx + 1:]):
                if tail == "S" or tail == "E":
                    tail_idx = start_idx + 1 + end_idx
                    label = region_dict[(start_idx, tail_idx)] if ((start_idx, tail_idx) in region_dict) else "O"
                    label_region.append(label)

                elif tail == "O":
                    break

    row_data_dict["entity_label"] = " ".join(label_region)
    if len(label_region):
        processed_data.append(row_data_dict)

In [73]:
len(processed_data)

5248

In [74]:
processed_data_df = pd.DataFrame(data=processed_data, columns=["text_a", "bd_label", "entity_label"])
processed_data_df.to_csv("dev.tsv", sep="\t",index=False)

In [75]:
processed_data_df

,text_a,bd_label,entity_label
0,对 儿 童 S A R S T 细 胞 亚 群 的 研 究 表 明 ， 与 成 人 S A ...,O O O B M M M M M E O O O O O O O O O B M M M ...,bod dis
1,研 究 证 实 ， 细 胞 减 少 与 肺 内 病 变 程 度 及 肺 内 炎 性 病 变 ...,O O O O O O O O O O S M M E O O O S M M M M E ...,bod sym bod sym
2,因 而 儿 童 肺 部 病 变 较 轻 ， 肺 内 炎 性 病 变 吸 收 较 快 。,O O O O B E M E O O O S M M M M E O O O O O,bod sym bod sym
3,一 般 有 明 确 S A R S 接 触 史 。,O O O O O B M M E O O O O,dis
4,（ 二 ） 临 床 症 状 、 体 征 与 成 年 人 类 似 ， 起 病 急 ， 主 要 ...,O O O O O O O O O O O O O O O O O O O O O O O ...,sym sym sym sym sym
...,...,...,...
5243,（ 5 ） 心 悸 伴 电 生 理 检 查 时 可 诱 发 持 续 性 室 上 性 心 动 ...,O O O B E O B M M M E O O O O B M M M M M M M ...,sym pro dis
5244,3 . 第 二 b 类 （ 1 ） 无 症 状 性 预 激 （ 心 电 图 显 示 W P ...,O O O O O O O O O O O O O O O O B M E O O O O ...,pro dis
5245,（ 2 ） 室 上 性 心 动 过 速 ， 年 龄 大 于 5 岁 ， R F C A 作 ...,O O O B M M M M M E O O O O O O O O B M M E O ...,dis pro dis dis
5246,（ 3 ） 室 上 性 心 动 过 速 ， 年 龄 小 于 5 岁 （ 包 括 婴 儿 ） ...,O O O B M M M M M E O O O O O O O O O O O O O ...,dis dis dru dru
